In [7]:
import urllib.request as ur
import re
from math import isnan

femPregDCT = ur.urlopen("https://raw.githubusercontent.com/AllenDowney/ThinkStats2/master/code/2002FemPreg.dct")

femRespDCT = ur.urlopen("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")

pregDataZip = "https://github.com/AllenDowney/ThinkStats2/blob/master/code/2002FemPreg.dat.gz?raw=true"

respDataZip = "https://github.com/AllenDowney/ThinkStats2/blob/master/code/2002FemResp.dat.gz?raw=true"


In [4]:
fPDCT = []

fRDCT = []

for i in femPregDCT:
	fPDCT.append(i.decode("utf-8") )
	
for i in femRespDCT:
	fRDCT.append(i.decode("utf-8") )

In [5]:
class FixedWidthVariables(object):
    """Represents a set of variables in a fixed width file."""
    def __init__(self, variables, index_base=0):
        """Initializes.
        variables: DataFrame
        index_base: are the indices 0 or 1 based?
        Attributes:
        colspecs: list of (start, end) index tuples
        names: list of string variable names
        """
        self.variables = variables
        # note: by default, subtract 1 from colspecs
        self.colspecs = variables[['start', 'end']] - index_base
        # convert colspecs to a list of pair of int
        self.colspecs = self.colspecs.astype(np.int).values.tolist()
        self.names = variables['name']

    def ReadFixedWidth(self, filename, **options):
        """Reads a fixed width ASCII file.
        filename: string filename
        returns: DataFrame
        """
        df = pd.read_fwf(filename,
                             colspecs=self.colspecs, 
                             names=self.names,
                             **options)
        return df

def ReadStataDct(dct_file):
	type_map = dict(byte=int, int=int, long=int, float=float, double=float)
	var_info = []
	for line in dct_file:
			match = re.search( r'_column\(([^)]*)\)', line)
			if match:
				start = int(match.group(1))
				t = line.split()
				vtype, name, fstring = t[1:4]
				name = name.lower()
				if vtype.startswith('str'):
					vtype = str
				else:
					vtype = type_map[vtype]
				long_desc = ' '.join(t[4:]).strip('"')
				var_info.append((start, vtype, name, fstring, long_desc))
	columns = ['start', 'type', 'name', 'fstring', 'desc']
	variables = pd.DataFrame(var_info, columns=columns)
	variables['end'] = variables.start.shift(-1)
	variables.loc[len(variables)-1, 'end'] = 0
	dct = FixedWidthVariables(variables, index_base=1)
	return dct

def CleanFemPreg(df):
    """Recodes variables from the pregnancy frame.
    df: DataFrame
    """
    # mother's age is encoded in centiyears; convert to years
    df.agepreg /= 100.0
    # birthwgt_lb contains at least one bogus value (51 lbs)
    # replace with NaN
    df.loc[df.birthwgt_lb > 20, 'birthwgt_lb'] = np.nan
    # replace 'not ascertained', 'refused', 'don't know' with NaN
    na_vals = [97, 98, 99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)
    df.hpagelb.replace(na_vals, np.nan, inplace=True)
    df.babysex.replace([7, 9], np.nan, inplace=True)
    df.nbrnaliv.replace([9], np.nan, inplace=True)
    # birthweight is stored in two columns, lbs and oz.
    # convert to a single column in lb
    # NOTE: creating a new column requires dictionary syntax,
    # not attribute assignment (like df.totalwgt_lb)
    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0    
    # due to a bug in ReadStataDct, the last variable gets clipped;
    # so for now set it to NaN
    df.cmintvw = np.nan

In [12]:
dct = ReadStataDct(fPDCT)
preg = dct.ReadFixedWidth(pregDataZip,compression='gzip' )
CleanFemPreg(preg)

In [13]:
dct = ReadStataDct(fRDCT)
resp = dct.ReadFixedWidth(respDataZip,compression='gzip' )

In [17]:
live = preg[preg.outcome == 1]
hist = {}
t = live.birthwgt_lb
for x in t:
	hist[x] = hist.get(x,0) + 1
	
clean_hist = {x: hist[x] for x in hist if not isnan(x)}

In [20]:
clean_hist

{8.0: 1889,
 7.0: 3049,
 9.0: 623,
 6.0: 2223,
 4.0: 229,
 5.0: 697,
 10.0: 132,
 12.0: 10,
 14.0: 3,
 3.0: 98,
 1.0: 40,
 11.0: 26,
 2.0: 53,
 13.0: 3,
 0.0: 8,
 15.0: 1}